# MIDS W207 Fall 2017 Final Project¶
## Data Set Up - Data Cleaning and Feature Engineering
Laura Williams, Kim Vignola, Cyprian Gascoigne  
SF Crime Classification

This notebook reads raw data (saved in a zip file) from Kaggle, processes and organizes the data for training a variety of machine learning models, and outputs the data as zipped csv files that other notebooks can unzip and use to train different models.

The intention is that data cleaning and/or feature engineering will be added to this file as we progress through the project and look for additional way to process the data to improve our predictions.

For ease of processing this data, exploratory data analysis will be in a separate notebook.

Single zipped output file (called data.zip) includes:  

1) train_data.csv and train_labels.csv - includes 80% of the total training data, for training models that are not yet going to be submitted to Kaggle

2) dev_data.csv and dev_labels.csv - includes 20% of the total training data, for testing models before they are submitted to Kaggle

3) train_data_all.csv and train_labels_all.csv - includes all the training data. After testing models with the train and dev data split above, train the model from this full set of data for submission to Kaggle.

4) test_data_all.csv - create predictions on this data for submission to Kaggle.

In [95]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import zipfile

In [96]:
# Unzip raw data into a subdirectory 
unzip_files = zipfile.ZipFile("raw_data.zip", "r")
unzip_files.extractall("raw_data")
unzip_files.close()

In [97]:
# Read CSV files into pandas dataframes
train = pd.read_csv("raw_data/train.csv")
test = pd.read_csv("raw_data/test.csv")
weather = pd.read_csv("raw_data/SF_county.csv")

In [98]:
# import datetime and holiday modules (note this takes a a few minutes to run)
from datetime import datetime, timedelta, date
import holidays

# extract month, year and hour from both datasets
train["month"] = train["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").month)
train["year"] = train["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").year)
train["day"] = train["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").day)
train["hour"] = train["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").hour)

test["month"] = test["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").month)
test["year"] = test["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").year)
test["day"] = test["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").day)
test["hour"] = test["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").hour)

# map holidays
US_Holidays = holidays.UnitedStates()
train["holidays"] = train["Dates"].map(lambda x: x in US_Holidays)
test["holidays"] = test["Dates"].map(lambda x: x in US_Holidays)

In [99]:
#from datetime import timedelta, date I Will mess around with this later
#print("2015-1-1" in US_Holidays)
#print(date(2015,1,1)in US_Holidays)
#print(date(train["year"], train["month"], 1) + timedelta(days = 1))

In [100]:
# create a dictionary for dayparts
time_periods = {6:"early_morning", 7:"early_morning", 8:"early_morning", 
               9:"late_morning", 10:"late_morning", 11:"late_morning",
              12:"early_afternoon", 13:"early_afternoon", 14:"early_afternoon",
              15:"late_afternoon", 16:"late_afternoon", 17:"late_afternoon",
              18:"early_evening",  19:"early_evening",  20:"early_evening",
              21:"late_evening", 22:"late_evening", 23:"late_evening",
              0:"late_night", 1:"late_night", 2:"late_night",
              3:"late_night", 4:"late_night", 5:"late_night"}

# map time periods
train["dayparts"] = train["hour"].map(time_periods)
test["dayparts"] = test["hour"].map(time_periods)

In [102]:
# clean up weather data
del weather['NAME']
weather["SNOW"] = weather["SNOW"].fillna(0)
print(weather)

          DATE  PRCP  SNOW  TMAX  TMIN
0       1/1/03  0.00   0.0    59    46
1       1/2/03  0.00   0.0    57    48
2       1/3/03  0.00   0.0    63    47
3       1/4/03  0.00   0.0    66    50
4       1/5/03  0.00   0.0    69    49
5       1/6/03  0.00   0.0    69    52
6       1/7/03  0.00   0.0    66    47
7       1/8/03  0.00   0.0    63    45
8       1/9/03  0.99   0.0    56    50
9      1/10/03  0.13   0.0    61    53
10     1/11/03  0.00   0.0    60    53
11     1/12/03  0.06   0.0    61    54
12     1/13/03  0.00   0.0    64    54
13     1/14/03  0.00   0.0    62    51
14     1/15/03  0.00   0.0    67    50
15     1/16/03  0.00   0.0    67    49
16     1/17/03  0.00   0.0    68    48
17     1/18/03  0.00   0.0    61    45
18     1/19/03  0.00   0.0    52    44
19     1/20/03  0.02   0.0    51    44
20     1/21/03  0.34   0.0    58    49
21     1/22/03  0.17   0.0    61    51
22     1/23/03  0.04   0.0    61    55
23     1/24/03  0.00   0.0    63    54
24     1/25/03  0.00   0.

In [103]:
# drop time from train and test date fields to be able to map Dates against weather data; remove hyphens too.
train["Dates"] = train["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
train["Dates"] = train["Dates"].map(lambda x: datetime.strftime(x,"%Y%m%d"))
test["Dates"] = test["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
test["Dates"] = test["Dates"].map(lambda x: datetime.strftime(x,"%Y%m%d"))

# Convert Weather DATE to same format as train and test data
weather["DATE"] = weather["DATE"].map(lambda x: datetime.strptime(x,"%m/%d/%y"))
weather["DATE"] = weather["DATE"].map(lambda x: datetime.strftime(x,"%Y%m%d"))

In [104]:
# confirm that dates are now in the same format
print(train["Dates"][0], train["Dates"].dtypes)
print(test["Dates"][0], test["Dates"].dtypes)
print(weather["DATE"][0], weather["DATE"].dtypes)

20150513 object
20150510 object
20030101 object


In [105]:
# convert date objects to numeric?  This didn't seem to do the job...still getting a string error.
train["Dates"] = pd.to_numeric(train["Dates"])
test["Dates"] = pd.to_numeric(test["Dates"])
weather["DATE"] = pd.to_numeric(weather["DATE"])
print(type(train["Dates"][0]))

<class 'numpy.int64'>


In [107]:
# When trying to map the dates getting error: 'str' object does not support item assignment

# loop over weather data and update weather variables according to dates
#for d in weather["DATE"]:
    #print(d)
   # if not train[train["Dates"]==d].empty:
   #     train[train["Dates"]==d]["rain"] = weather[weather["DATE"]==d].PRCP
   #     train[train["Dates"]==d]["max_temp"] = weather[weather["DATE"]==d].TMAX
   #     train[train["Dates"]==d]["min_temp"] = weather[weather["DATE"]==d].TMIN
    
    #if not test[test["Dates"]==d].empty:
  #      test[test["Dates"]==d]["rain"] = weather[weather["DATE"]==d].PRCP
      #  test[test["Dates"]==d]["max_temp"] = weather[weather["DATE"]==d].TMAX
     #   test[test["Dates"]==d]["min_temp"] = weather[weather["DATE"]==d].TMIN

        
weather_train = pd.merge(train, weather, how='left', left_on="Dates", right_on = "DATE")
weather_test = pd.merge(test, weather, how='left', left_on="Dates", right_on = "DATE")
print(weather_train)

           Dates                Category  \
0       20150513                WARRANTS   
1       20150513          OTHER OFFENSES   
2       20150513          OTHER OFFENSES   
3       20150513           LARCENY/THEFT   
4       20150513           LARCENY/THEFT   
5       20150513           LARCENY/THEFT   
6       20150513           VEHICLE THEFT   
7       20150513           VEHICLE THEFT   
8       20150513           LARCENY/THEFT   
9       20150513           LARCENY/THEFT   
10      20150513           LARCENY/THEFT   
11      20150513          OTHER OFFENSES   
12      20150513               VANDALISM   
13      20150513           LARCENY/THEFT   
14      20150513            NON-CRIMINAL   
15      20150513            NON-CRIMINAL   
16      20150513                 ROBBERY   
17      20150513                 ASSAULT   
18      20150513          OTHER OFFENSES   
19      20150513            NON-CRIMINAL   
20      20150513           LARCENY/THEFT   
21      20150513                

Next, remove outliers.

In [11]:
# Data indicates outliers with latitude = 90 (aka the North Pole). Test data has these same outliers.
train.Y.describe()

count    878049.000000
mean         37.771020
std           0.456893
min          37.707879
25%          37.752427
50%          37.775421
75%          37.784369
max          90.000000
Name: Y, dtype: float64

In [12]:
# remove cells where latitude > 40
train = train[train.Y < 38]
test = test[test.Y < 38]

In [13]:
# print new shape
print(train.shape)
print(test.shape)

print("Cases removed from train data =", np.sum(878049 - train.shape[0]))
print("Cases removed from test data =", np.sum(884262 - test.shape[0]))

(877982, 17)
(884186, 15)
Cases removed from train data = 67
Cases removed from test data = 76


In [14]:
print(train.columns)
print(test.columns)

Index(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y', 'month', 'year', 'hour', 'holidays',
       'dayparts', 'rain', 'max_temp', 'min_temp'],
      dtype='object')
Index(['Id', 'Dates', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'month',
       'year', 'hour', 'holidays', 'dayparts', 'rain', 'max_temp', 'min_temp'],
      dtype='object')


In [23]:
# Encode string features into numeric features
LE = preprocessing.LabelEncoder()

train_data_all = np.column_stack((LE.fit_transform(train['Dates']),
                                 LE.fit_transform(train['DayOfWeek']),
                                 LE.fit_transform(train['PdDistrict']),
                                 LE.fit_transform(train['Address']),
                                 train['X'],
                                 train['Y']))

train_labels_all = np.array(train['Category'])

test_data_all = np.column_stack((LE.fit_transform(test['Dates']),
                                LE.fit_transform(test['DayOfWeek']),
                                LE.fit_transform(test['PdDistrict']),
                                LE.fit_transform(test['Address']),
                                test['X'],
                                test['Y']))


In [16]:
# Shuffle data and set aside 20% as development data
n = train_data_all.shape[0]

shuffle = np.random.permutation(np.arange(train_data_all.shape[0]))

train_data_all = train_data_all[shuffle]
train_labels_all = train_labels_all[shuffle]

n_train = int(0.8*n)

train_data = train_data_all[:n_train,:]
train_labels = train_labels_all[:n_train]
dev_data = train_data_all[n_train:,:]
dev_labels = train_labels_all[n_train:]

In [20]:
# print shapes and some data to compare before and after csv conversion
print("train_data shape is", train_data.shape)
print("train_labels shape is", train_labels.shape)
print("dev_data shape is", dev_data.shape)
print("dev_labels shape is", dev_labels.shape)
print("train_data_all shape is", train_data_all.shape)
print("train_data_all shape is", train_data_all.shape)
print("train_labels_all shape is", train_labels_all.shape)
print("test_data_all shape is", test_data_all.shape)

train_data shape is (702439, 6)
train_labels shape is (702439,)
dev_data shape is (175610, 6)
dev_labels shape is (175610,)
train_data_all shape is (878049, 6)
train_data_all shape is (878049, 6)
train_labels_all shape is (878049,)
test_data_all shape is (884262, 6)


In [18]:
# Save arrays as CSV files in a subdirectory

# NOTE: mkdir will make a "csv" directory in your local repo if there is not already one there.
# It will return an error if the directory already exists in your local repo
# but that will not impact how this code runs

! mkdir csv
np.savetxt("csv/train_data.csv", train_data, delimiter=",")
np.savetxt("csv/train_labels.csv", train_labels, fmt="%s", delimiter=",")
np.savetxt("csv/dev_data.csv", dev_data, delimiter=",")
np.savetxt("csv/dev_labels.csv", dev_labels, fmt="%s", delimiter=",")
np.savetxt("csv/train_data_all.csv", train_data_all, delimiter=",")
np.savetxt("csv/train_labels_all.csv", train_labels_all, fmt="%s", delimiter=",")
np.savetxt("csv/test_data_all.csv", test_data_all, delimiter=",")

mkdir: csv: File exists


In [19]:
# Zip up the CSV files

# **IMPORTANT**  This code will rewrite the existing data.zip file in your local repo
# You will need to push it to the group repo for everyone to have the updated zip file

zip_files = zipfile.ZipFile("data.zip", "w")
zip_files.write("csv/train_data.csv", compress_type=zipfile.ZIP_DEFLATED)
zip_files.write("csv/train_labels.csv", compress_type=zipfile.ZIP_DEFLATED)
zip_files.write("csv/dev_data.csv", compress_type=zipfile.ZIP_DEFLATED)
zip_files.write("csv/dev_labels.csv", compress_type=zipfile.ZIP_DEFLATED)
zip_files.write("csv/train_data_all.csv", compress_type=zipfile.ZIP_DEFLATED)
zip_files.write("csv/train_labels_all.csv", compress_type=zipfile.ZIP_DEFLATED)
zip_files.write("csv/test_data_all.csv", compress_type=zipfile.ZIP_DEFLATED)
zip_files.close()